Question: Where are the worst and most liveable places in Nigeria?

to answer this, I will:
analyse a rich data about Nigeria
analyse what was said about those places

the data to be analyzed is wikipedia articles gotten from the wikipedia dump.
Wikipedia, because it contains a lot of information about different places 

1. Getting the data from the Wikipedia dump, parsing it and saving only articles about...

2. Merging the articles together in one csv file
The articles were in json format. so next step is to convert it to csv.
what is now loaded here is the coverted file. Now in csv format, ready to be loaded into pandas dataframe

In [ ]:
#importing the necessary modules

import pandas as pd
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")

In [ ]:
#what is now loaded here is the csv file that contains articles that mention Nigeria
#a total of 55474 rows and 3 columns: 'id', 'title', 'text'


wikidump = pd.read_csv(r'Desktop\mergedDump.csv')
wikidump

3. Getting the location in the title column of all the rows. So I can know what exact location the article is about

In [ ]:
#using spacy to get named entities in the title column
#the found locations are set in a new column 'location'

wikidump['text'] = wikidump['text'].astype(str) #cast type so the below function doesn't encounter any error. as spacy works
                                                #on only string or Doc

def get_loc(text):
    doc = nlp(text)
    results = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    return results

wikidump["location"] = wikidump['title'].apply(get_loc)

In [ ]:
#I wrote the result to a new csv to save the time of running entity recognition everytime

wikidump = pd.read_csv(r'Desktop\wikidump-with-location.csv', index_col = 0)


wikidump["location"] = wikidump["location"].apply(eval)  #this removes the quotation marks on the items in the location list

wikidump['length'] = wikidump.location.apply(len) #to get the number of locations in each row of the new location column

In [ ]:
#to remove rows where the location is more or less than 1. so as to get rows(articles) that specially describe just one place

wikidump = wikidump.loc[wikidump['length'] == 1] 
wikidump

del wikidump['length'] #delete the length column. It is no longer needed as the filtering has now been done

wikidump.reset_index(drop=True)


#there are now 8444 rows × 4 columns left

In [ ]:
wikidump = wikidump[wikidump["title"].str.contains("Category:") == False] #cleaning some irrelevant rows
wikidump.reset_index(drop=True)

#there are now 5081 rows × 4 columns

4. Tokenizing the sentences in the articles. ie the 'text' column

In [ ]:
nlp.add_pipe('sentencizer')

def sent_token(text):
    doc = nlp(text)
    results= [sent.text for sent in doc.sents]
    return results

wikidump['sentences'] = wikidump['text'].astype(str).apply(sent_token)

In [ ]:
#I wrote the result to a new csv to save the time of running sentence tokenization everytime

wikidump = pd.read_csv(r'Documents\wiki_sent_token.csv', index_col = 0)

wikidump["sentences"] = wikidump["sentences"].apply(eval) #this removes the quotation marks on the items in the sentences list


wikidump = wikidump.explode('sentences') #separate the sentences into diferent rows
wikidump = wikidump.reset_index(drop=True)

#there are now 481795 rows × 5 columns

In [ ]:
#some extra cleaning, removing 'irrelevant' rows

wikidump = wikidump[wikidump["sentences"].str.contains("==") == False].reset_index(drop=True) 
wikidump = wikidump[wikidump["sentences"].str.contains("\* ") == False].reset_index(drop=True)

#there are now 358647 rows × 5 columns

In [ ]:
del wiki['location']  #delete this location column. It is no longer needed
wiki.reset_index(drop=True)

5. Getting the location in each row. 

In [ ]:
wiki['sentences'] = wiki['sentences'].astype(str)

def get_loc(text):
    doc = nlp(text)
    results = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    return results

wiki["location"] = wiki['sentences'].apply(get_loc)


#written to 
wiki = pd.read_csv(r'Documents\wikipedia.csv')